# Python para data science na prática

---
# Gerenciando dados ausentes com pandas

## Introdução

Dados ausentes geralmente ocorrem em análises de dados. O Pandas simplifica ao máximo o trabalho com dados ausentes. Por exemplo, todas as estatísticas descritivas de objetos pandas excluem dados ausentes por padrão. O pandas usa o valor de ponto flutuante (não é um número) para que os dados numéricos representem os dados ausentes. `NaN`

Métodos para manipular objetos:NA

| Argumento | Descrição |
|:----------:|:------------|
| drpna()    | Filtra rótulos de eixo com base no fato de os valores de cada rótulo terem dados ausentes, com limites diferentes para a quantidade de dados ausentes a serem tolerados |
| fillna()     | Preenche os dados ausentes com um valor ou com um método de interpolação, como ou `ffill` ou `bfill` |
| isna()       | Retorna valores booleanos indicando quais valores estão faltando/`NA` |
|notna()      | Retorna todos os valores não ausentes nos dados |

Este notebook apresenta algumas maneiras de gerenciar dados ausentes usando Pandas DataFrames.

In [1]:
# importar bibliotecas
import pandas as pd


## Leitura dos dados (dataset)

In [ ]:
# dataset de covid-19 de 2024
df_esus = pd.read_csv('data/esus2024.csv', sep=';', on_bad_lines='skip', index_col=False, dtype='unicode')
df_esus.head()

In [ ]:
# dimensions
df_esus.shape

In [ ]:

df = pd.read_csv(
    "https://raw.githubusercontent.com/kjam/data-cleaning-101/master/data/iot_example_with_nulls.csv"
)
df.head()

In [ ]:
df.shape

## Verificando os dados

No pandas, uma convenção emprestada da linguagem de programação R foi adotada e os dados ausentes foram chamados de `NA`, que significa *não disponível*. Em aplicações estatísticas, os dados podem ser dados que não existem ou dados que existem, mas não foram observados (por exemplo, devido a problemas na coleta de dados). 

Ao limpar os dados para análise, muitas vezes é *importante analisar os próprios dados ausentes para identificar problemas na coleta de dados ou possíveis vieses nos dados devido a os dados em falta*. Fonte [Missing data](https://www.python4data.science/en/24.1.0/clean-prep/nulls.html)

Primeiro, exibimos os primeiros 10 registros de dados:

In [ ]:
# mostra os 20 registros
df_esus.head(10)

In [ ]:
df.head(10)

Verificar qual tipo de dados têm as colunas

In [ ]:
# tipos
df_esus.dtypes

In [ ]:
df.dtypes

Também se pode exibir os valores e sua frequência, por exemplo, para uma coluna específica.

In [ ]:
# conta os valores das colunas
#df_esus.qualAntiviral.value_counts()
df_esus.classificacaoFinal.value_counts()

In [ ]:
df.note.value_counts()

## Remover todos os valores nulos (incluindo a indicação n/a)

### Usando panas.read_csv

`pandas.read_csv` geralmente já filtra muitos valores que reconhece como `NA`ou `NAN`. Outros valores podem ser especificados com `na_values`

In [ ]:
# caso 1
df_esus1 = pd.read_csv('data/esus2024.csv', sep=';', on_bad_lines='skip', index_col=False, dtype='unicode', na_values=["n/a"])
df_esus1.head()

In [ ]:
df_esus1.shape

In [ ]:
# caso 2
df2 = pd.read_csv(
    "https://raw.githubusercontent.com/kjam/data-cleaning-101/master/data/iot_example_with_nulls.csv",
    na_values=["n/a"],)
df2.head()

### Usando `pandas.DataFrame.dropna`

Valores ausentes podem ser excluídos com pandas `DataFrame.dropna`.

Para analisar a extensão das exclusões, exibimos a extensão do DataFrame antes e depois da exclusão com pandas `DataFrame.shape`

In [ ]:
#caso 1 e 2: antes
print('Caso 1:',df_esus1.shape)
print('caso 2:',df2.shape)

In [ ]:
# caso 1 e 2 : depois
c1 = df_esus1.dropna()
c2 = df2.dropna()
print('Caso 1:',c1.shape)
print('caso 2:',c2.shape)

Caso 1: Perderiamos todos os registros.
Caso 2: perderíamos mais de 2/3 dos registros com `pandas.DataFrame.dropna`

Analisar se as linhas ou colunas têm dados ausentes ou faltantes. Se pode remover linhas ou colunas que não contêm valores.

Las linhas/colunas vazias devem ser removidas `how=all` ou `axis=0` (linha) ou `axis=1` (coluna)

In [ ]:
# caso 1: colunas
esus_col = df_esus.dropna(how="all", axis=1).shape
esus_col

In [ ]:
#caso 2
df_col = df.dropna(how='all', axis=1).shape
df_col

## Encontre todas as colunas onde todos os dados estão presentes

In [ ]:
complete_columns_c1 = list(df_esus.columns)
complete_columns_c1

In [ ]:
len(complete_columns_c1)

In [ ]:
complete_columns_c2 = list(df.columns)
complete_columns_c2

## Encontre todas as colunas onde há mais dados disponíveis

In [ ]:
list(df_esus.dropna(thresh=int(df_esus.shape[0] * 0.9), axis=1).columns)

In [ ]:
# Caso 2
list(df.dropna(thresh=int(df.shape[0] * 0.9), axis=1).columns)

## Encontre todas as colunas onde faltam dados

Usando `pandas.DataFrame.isnull` se pode encontrar valores ausentes e com `pandas.DataFrame.any` se pode saber se um elemento é válido, geralmente por meio de uma coluna.

In [37]:
# caso 1
incomplete_columns_c1 = list(df_esus.columns[df_esus.isnull().any()])

In [ ]:
# caso 1
incomplete_columns_c1

In [ ]:
len(incomplete_columns_c1)

In [39]:
# caso 2
incomplete_columns_c2 = list(df.columns[df.isnull().any()])

In [ ]:
# caso 2
incomplete_columns_c2

Se pode gerar o número de valores ausentes por coluna usando `num_missing`

In [ ]:
# Caso 1
for col in incomplete_columns_c1:
    num_missing = df_esus[df_esus[col].isnull() == True].shape[0]
    print(f"numero de valores faltantes por coluna {col}: {num_missing}")

In [ ]:
# Caso 1
for col in incomplete_columns_c2:
    num_missing = df[df[col].isnull() == True].shape[0]
    print(f"numero de valores faltantes por coluna {col}: {num_missing}")

Se pode gerar esses valores como uma porcentagem

In [ ]:
# caso 1
for col in incomplete_columns_c1:
    percent_missing = df_esus[df_esus[col].isnull() == True].shape[0] / df_esus.shape[0]
    print(f"porcentagem de valores faltantes por coluna {col}: {percent_missing}")

In [ ]:
# caso 2
for col in incomplete_columns_c2:
    percent_missing = df[df[col].isnull() == True].shape[0] / df.shape[0]
    print(f"porcentagem de valores faltantes por coluna {col}: {percent_missing}")

## Substituir os dados ausentes

Para poder verificar as alterações na coluna `caso 1:cns` e `caso2:latest` se usa pandas. `Series.value_counts`. O método retorna uma série contendo o número de valores exclusivos:

In [ ]:
# caso 1
df_esus.cns.value_counts()

In [ ]:
# Caso 2
df.latest.value_counts()

Substituir os valores ausentes na coluna `cns:0` e `latest:0` com `DataFrame.fillna`

In [ ]:
#Caso 1
df_esus.cns = df_esus.cns.fillna(0)
df_esus.cns.value_counts()

In [ ]:
# caso 2
df.latest = df.latest.fillna(0)
df.latest.value_counts()

---
Python para data science &copy; Jorge Zavaleta, 2024